In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


In [2]:
psf = 'inp_files/da.psf'
pdb = 'inp_files/smd_ini.pdb'
prm = 'inp_files/par_all27_prot_lipid_cmap.prm'

In [232]:
def com_vel(v, mas):
    return (np.sum(mas * v, axis = 0) / np.sum(mas))

In [242]:
def KE(v, mas):
    return ((0.5 / na) * np.sum(mas * v**2))

In [244]:
alpha = (3 * kb * T) / (2 * KE(initial_velocities, mass))

In [246]:
initial_velocities *= alpha

In [247]:
initial_velocities -= com_vel(initial_velocities, mass)

In [3]:
import parmed

In [4]:
from parmed.charmm import CharmmParameterSet

In [5]:
prm = CharmmParameterSet("inp_files/par_all27_prot_lipid_cmap.prm")

/home/shaunak/anaconda3/envs/MD/lib/python3.6/site-packages/parmed/charmm/parameters.py:747: ParameterWarning: Atom type C not present in AtomType list
  warnings.warn('Atom type %s not present in AtomType list' % key, ParameterWarning)


In [9]:
from parmed.formats.registry import load_file

In [21]:
psf = load_file('inp_files/da.psf')

# Alternatively this is also okay
# from parmed.charmm import CharmmPsfFile
# psf = CharmmPsfFile('inp_files/da.psf')
psf.bonds

TrackedList([
	<Bond <Atom N [0]; In ALA 0>--<Atom CA [3]; In ALA 0>; type=None>
	<Bond <Atom HT2 [1]; In ALA 0>--<Atom N [0]; In ALA 0>; type=None>
	<Bond <Atom HT3 [2]; In ALA 0>--<Atom N [0]; In ALA 0>; type=None>
	<Bond <Atom CA [3]; In ALA 0>--<Atom HA [4]; In ALA 0>; type=None>
	<Bond <Atom CB [5]; In ALA 0>--<Atom CA [3]; In ALA 0>; type=None>
	<Bond <Atom CB [5]; In ALA 0>--<Atom HB1 [6]; In ALA 0>; type=None>
	<Bond <Atom CB [5]; In ALA 0>--<Atom HB2 [7]; In ALA 0>; type=None>
	<Bond <Atom CB [5]; In ALA 0>--<Atom HB3 [8]; In ALA 0>; type=None>
	<Bond <Atom C [9]; In ALA 0>--<Atom CA [3]; In ALA 0>; type=None>
	<Bond <Atom C [9]; In ALA 0>--<Atom N [11]; In ALA 1>; type=None>
	<Bond <Atom O [10]; In ALA 0>--<Atom C [9]; In ALA 0>; type=None>
	<Bond <Atom N [11]; In ALA 1>--<Atom HN [12]; In ALA 1>; type=None>
	<Bond <Atom N [11]; In ALA 1>--<Atom CA [13]; In ALA 1>; type=None>
	<Bond <Atom CA [13]; In ALA 1>--<Atom HA [14]; In ALA 1>; type=None>
	<Bond <Atom CB [15]; In ALA 1>

In [22]:
pdb = load_file('inp_files/smd_ini.pdb')

In [211]:
from copy import deepcopy

class Energies:
    def __init__(self, psf, pdb, prm):
        self.psf = psf
        self.pdb = pdb
        self.prm = prm
        
        ## initializing bond types and coordinates
        coords = pdb.coordinates
        bonds = psf.bonds
        angles = psf.angles
        dihedrals = psf.dihedrals
        impropers = psf.impropers
        
        for bond in bonds:
            ## bond types
            tid1 = bond.atom1.type
            tid2 = bond.atom2.type
            
            t = (tid1, tid2)
            bond.type = prm.bond_types[t]
    
            ## coordinates
            b1 = bond.atom1
            b1.xx, b1.xy, b1.xz = coords[b1.idx]
            
            b2 = bond.atom2
            b2.xx, b2.xy, b2.xz = coords[b2.idx]
            
        ## initializing angle types
        for angle in angles:
            a1 = angle.atom1.type
            a2 = angle.atom2.type
            a3 = angle.atom3.type
            
            t = (a1, a2, a3)
            
            angle.type = prm.angle_types[t]
            a = angle.measure()
            a0 = angle.type
            
        for i, dihedral in enumerate(dihedrals):
            a1 = dihedral.atom1.type
            a2 = dihedral.atom2.type
            a3 = dihedral.atom3.type
            a4 = dihedral.atom4.type
            
            t = (a1, a2, a3, a4)
            g = self._get_bond_type(t, prm.dihedral_types)
            dihedrals[i].type = prm.dihedral_types[g]
            
        for i, improper in enumerate(impropers):
            a1 = improper.atom1.type
            a2 = improper.atom2.type
            a3 = improper.atom3.type
            a4 = improper.atom4.type
            
            t = (a1, a2, a3, a4)
            g = self._get_bond_type(t, prm.improper_types)
            impropers[i].type = prm.improper_types[g]
        
    def bond(self):
        bonds = psf.bonds
        s = 0
        for bond in bonds:
            r1 = pdb.coordinates[bond.atom1.idx]
            r2 = pdb.coordinates[bond.atom2.idx]
            k = bond.type.k
            dist = np.linalg.norm(r1 - r2)
            
            r0 = bond.type.req
            cust_energy = k * ((dist - r0)**2)
            s += cust_energy
            
        return s    
    
    def angle(self):
        s = 0
        
        for angle in psf.angles:
            theta = angle.measure()
            theta0 = angle.type.theteq
            k = angle.type.k
            theta_diff = (np.pi * (theta - theta0)) / 180
            en = k * (theta_diff ** 2)
#             print(en, angle.energy())
            s += en
        return s
    
    ## DIHEDRAL AND IMPROPER HELPER FUNCTION
    def _get_bond_type(self, t, search_list):
        '''
        Input :
        t : a 4-atom tuple, 
        search_list: usually prm.dihedral_types or prm.improper_types

        Return type: BondType
        '''
        for dtype in search_list:
            val = True
            for i in range(4):
                val = val and (dtype[i] == t[i] or dtype[i] == 'X')

            if val:
                return dtype

            ## Attempt reverse search
            val = True
            for i in range(4):
                val = val and (dtype[i] == t[3 - i] or dtype[i] == 'X')

            if val:
                return dtype
        return None
    
    def dihedral(self):
        dihedrals = psf.dihedrals
        for dihed in dihedrals:
            dtype = dihed.type
            
            total_energy = 0
            chi = dihed.measure() * (np.pi / 180)
            if isinstance(dtype, parmed.DihedralType):
                n = dtype.per
                kchi = dtype.phi_k
                phase = dtype.phase * (np.pi / 180)
                en = kchi * (1 + np.cos(n * chi - phase))
                total_energy += en
                
            elif isinstance(dtype, parmed.DihedralTypeList):
                e = 0
                for term in dtype:
                    n = term.per
                    kchi = term.phi_k
                    phase = term.phase * (np.pi / 180)
                    en = kchi * (1 + np.cos(n * chi - phase))
                    e += en
                e *= 0.5
                total_energy += e
            else:
                printf("Cant determine parameters for {}".format(t))
                return 0
        return total_energy
    
    def improper(self):
        impropers = psf.impropers
        total_energy = 0
        for imp in impropers:
            psi = imp.measure() * (np.pi / 180)
            dtype = imp.type
            psi_0 = dtype.psi_eq * (np.pi / 180)
            k = dtype.psi_k 
            en = k *(psi - psi_0)**2
            
            total_energy += en
            
        return total_energy
    
    # NON BONDED
    def LJ(self):
#         print(psf.__dir__())
#         print(prm.nbfix_types)
        for i in prm.nbfix_types:
            print(i)


In [212]:
h = Energies(psf, pdb, prm)
h.LJ()

In [218]:
pr = parmed.parameters.ParameterSet()

In [221]:
print(pr.nbfix_types)

OrderedDict()


In [224]:
pr = parmed.charmm.parameters.CharmmParameterSet.read_parameter_file(pfile = "inp_files/par_all27_prot_lipid_cmap.prm")

TypeError: read_parameter_file() missing 1 required positional argument: 'self'